# Model 9

First fracture prediction model.  Convolutional downsampling, residual blocks, global pooling before class prediction. 

Batch size 1 to accomodate memory constraints AND different size samples.

Model quickly learns to classify everything as non-fracture.

## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, MaxPooling3D, GlobalMaxPooling3D,
                          Conv3D, UpSampling3D, BatchNormalization, Concatenate, Add)
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
importlib.reload(util)
import preprocessing
importlib.reload(preprocessing)
import datagen
importlib.reload(datagen)
import modelutil
importlib.reload(modelutil)

SEED = 0
EPOCHS = 100
BATCH_SIZE = 1
PATCH_SHAPE = (32, 32, 32)

MODEL_NAME = 'model_09'

DATA_DIR = Path('/data2').expanduser()
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'

MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard' / MODEL_NAME
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()

%load_ext autoreload
%autoreload 2

## Data Generation

In [ ]:
train_gen, val_gen = datagen.get_nifti_fracture_datagens(
    preprocessed_metadata_path=PP_MD_PATH, batch_size=BATCH_SIZE, seed=SEED)


## Build Model

Downsampled agressively with strided convolutions to fit in memory.  Residual blocks.  Global pooling at the end to classify.


In [ ]:
def build_residual_encoder_decoder_block(x, n_a, n_d=1, use_bn=True):

    x = batchnorm_conv_block(x, n_a, use_bn=use_bn)
    
    if n_d > 0:
        x_e = x # shape: (32, 32, 32, 16)
        x_e = MaxPooling3D(padding='same')(x_e) # shape: (16, 16, 16, 16)
        x_e = build_residual_encoder_decoder_block(x_e, n_a, n_d - 1, use_bn=use_bn) # recursive call
        x_d = UpSampling3D()(x_e) # shape (32, 32, 32, 16)
        x = Concatenate()([x, x_d]) # residual join.  shape (32, 32, 32, 32)
        x = batchnorm_conv_block(x, n_a, use_bn=use_bn)
    
    return x


def batchnorm_conv_block(x, n_a, use_bn=True):
    if use_bn:
        x = BatchNormalization()(x)
        
    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # shape: (32, 32, 32, 1) = 32768
    return x


def build_residual_block(x, n_a, n_l=1, use_bn=True):
    '''
    n_l: number of layers/convolutions in the residual path.
    '''
    x_r = x
    for i in range(n_l):
        x_r = batchnorm_conv_block(x_r, n_a, use_bn=use_bn)
        
    x = Add()([x, x_r])  
    return x


def build_downsampling_conv_block(x, n_a, use_bn=True):
    if use_bn:
        x = BatchNormalization()(x)
        
    x = Conv3D(n_a, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same', activation='relu')(x) 
    return x
    
    
def build_model(input_shape, n_a=16, n_r=2, n_d=4, use_bn=True):
    '''
    3D convolutional autoencoder that treats u-net architecture as a residual block.
    
    1 poolings reduce input from shape to shape/2, which in 3d is 1/8th the size of the original shape,
    a very respectable compression factor.
    '''

    x_input = Input(shape=input_shape)
    x = x_input

    x = build_downsampling_conv_block(x, n_a=n_a*2, use_bn=use_bn)
    x = build_residual_block(x, n_a=n_a*2, n_l=1, use_bn=use_bn) 

    # u-net
#    x = build_residual_encoder_decoder_block(x, n_a=(n_a//2), n_d=n_d)
    
    # upsample for autoencoder
#     x_ae = UpSampling3D()(x)
#     x_ae = batchnorm_conv_block(x_ae, n_a=n_a)
#     y_ae = Conv3D(1, kernel_size=(3, 3, 3), padding='same', activation='sigmoid')(x)

    x = build_downsampling_conv_block(x, n_a=n_a*4, use_bn=use_bn)
    x = build_residual_block(x, n_a=n_a*4, n_l=1, use_bn=use_bn) 

    x = build_downsampling_conv_block(x, n_a=n_a*8, use_bn=use_bn)
    x = build_residual_block(x, n_a=n_a*8, n_l=1, use_bn=use_bn) 

    x = build_downsampling_conv_block(x, n_a=n_a*16, use_bn=use_bn)
    x = build_residual_block(x, n_a=n_a*16, n_l=1, use_bn=use_bn) 

    x = build_downsampling_conv_block(x, n_a=n_a*32, use_bn=use_bn)
    x = build_residual_block(x, n_a=n_a*32, n_l=1, use_bn=use_bn) 
    x = build_residual_block(x, n_a=n_a*32, n_l=1, use_bn=use_bn)
     
    # pool and predict
    x = GlobalMaxPooling3D()(x)
    if use_bn:
        x = BatchNormalization()(x)
        
    x = Dense(n_a*16, activation='relu')(x)
    y_frac = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=x_input, outputs=y_frac)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
   
   

In [ ]:
model = build_model(input_shape=(None, None, None, 1,), n_a=4, n_r=4, n_d=4, use_bn=False)

In [ ]:
print(model.summary())


In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))


## Train and Evaluate Model

- Add callbacks to save model every 20 epochs and to log performance stats every epoch, so we have the results saved somewhere for charting.


In [ ]:
# history, log_path = modelutil.train_model(
#     model, train_gen, val_gen, epochs=40, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, model_name=MODEL_NAME, 
#     log_dir=LOG_DIR, tensorboard_log_dir=TENSORBOARD_LOG_DIR, max_queue_size=20, use_multiprocessing=True, 
#     class_weight={0: 1, 1: 5})
history, log_path = modelutil.train_model_epoch(train_gen, val_gen, epoch=40, epochs=200, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, model_name=MODEL_NAME, 
    log_dir=LOG_DIR, tensorboard_log_dir=TENSORBOARD_LOG_DIR, max_queue_size=20, use_multiprocessing=True, 
    class_weight={0: 1, 1: 5})

## Visualize Training Progress

In [ ]:
# read metrics from the log file
# log_path = LOG_DIR / (model_name + '_2018-04-26T17:29:02.902740_log.csv')
# log_path = Path('/data2/uvm_deep_learning_project/log/model_09_2018-04-28T02:02:18.169239_log.csv')
metrics = pd.read_csv(log_path)

In [ ]:
print(pd.concat([metrics[::10], metrics[-1:]])) # every 10th metric and the last one

In [ ]:
# Plot Training and Validation Accuracy 
axes = plt.gca()
axes.set_ylim([0.0,1.0]) # Show results on 0..1 range
plt.plot(metrics["acc"])
plt.plot(metrics["val_acc"])
plt.legend(['Training Accuracy', "Validation Accuracy"])
plt.show()

# Plot Training and Validation Loss
plt.plot(metrics["loss"])
plt.plot(metrics["val_loss"])
plt.legend(['Training Loss', "Validation Loss"])
plt.show()



### Confusion Matrix Results Over Time

Visualize how the results of the model improve over time.


In [ ]:
# confusion_matrix_by_epochs()
modelutil.confusion_matrix_by_epochs(MODELS_DIR, MODEL_NAME, [1, 10, 200], val_gen)
    